# [Create Agent With OpenAPI](https://learn.microsoft.com/en-us/python/api/overview/azure/ai-agents-readme?view=azure-python-preview#create-agent-with-openapi)
AzureFunctionTool contains the input and output queues of azure function and the description of input parameters.

Inspired by [sample_agents_openapi.py](https://github.com/Azure/azure-sdk-for-python/blob/azure-ai-projects_1.0.0b4/sdk/ai/azure-ai-projects/samples/agents/sample_agents_openapi.py)

# Constants

In [1]:
import os
from IPython.display import Markdown, display
from dotenv import load_dotenv # requires python-dotenv
from PIL import Image # requires pip install pillow
from datetime import datetime
from common.agents_helper_functions_NEW import *
import importlib.metadata
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

if not load_dotenv("./../config/credentials_my.env"):
    print("Environment variables not loaded, cell execution stopped")
else:
    print("Environment variables have been loaded ;-)")

project_endpoint = os.environ["PROJECT_ENDPOINT"]
deployment_name =  os.environ["MODEL_DEPLOYMENT_NAME"]
api_version = os.environ["OPENAI_API_VERSION"] # at least "2025-03-01-preview"

print(f'Project Endpoint: <{project_endpoint}>')
print(f"azure-ai-projects library installed version: {importlib.metadata.version("azure-ai-projects")}")
print(f"azure-ai-agents library installed version: {importlib.metadata.version("azure-ai-agents")}")

Environment variables have been loaded ;-)
Project Endpoint: <https://mmai-swc-new01-prj-resource.services.ai.azure.com/api/projects/mmai-swc-new01-prj>
azure-ai-projects library installed version: 1.0.0b11
azure-ai-agents library installed version: 1.1.0b1


# Create AI Foundry Agents Client

In [2]:
from azure.ai.agents import AgentsClient
from azure.identity import DefaultAzureCredential

agents_client = AgentsClient(
    endpoint=project_endpoint,
    credential=DefaultAzureCredential(),
)

agents_client

# Load Weather OpenAPI

In [3]:
import jsonref

with open("./openapi/weather_openapi.json", "r") as f:
    openapi_spec = jsonref.loads(f.read())

openapi_spec

{'openapi': '3.1.0',
 'info': {'title': 'get weather data',
  'description': 'Retrieves current weather data for a location based on wttr.in.',
  'version': 'v1.0.0'},
 'servers': [{'url': 'https://wttr.in'}],
 'auth': [],
 'paths': {'/{location}': {'get': {'description': 'Get weather information for a specific location',
    'operationId': 'GetCurrentWeather',
    'parameters': [{'name': 'location',
      'in': 'path',
      'description': 'City or location to retrieve the weather for',
      'required': True,
      'schema': {'type': 'string'}},
     {'name': 'format',
      'in': 'query',
      'description': 'Always use j1 value for this parameter',
      'required': True,
      'schema': {'type': 'string', 'default': 'j1'}}],
    'responses': {'200': {'description': 'Successful response',
      'content': {'text/plain': {'schema': {'type': 'string'}}}},
     '404': {'description': 'Location not found'}},
    'deprecated': False}}},
 'components': {'schemes': {}}}

# Create Auth object for the OpenApiTool 
## Note: connection or managed identity auth setup requires additional setup in Azure

In [4]:
from azure.ai.agents.models import OpenApiAnonymousAuthDetails

auth = OpenApiAnonymousAuthDetails()
auth

{'type': 'anonymous'}

# Initialize agent OpenApi tool using the read in OpenAPI spec

In [5]:
from azure.ai.agents.models import OpenApiTool

openapi = OpenApiTool(
    name="get_weather", spec=openapi_spec, description="Retrieve weather information for a location", auth=auth
)
print(f"openapi.definitions: {openapi.definitions}")
print(f"\nopenapi.resources: {openapi.resources}")

openapi.definitions: [{'type': 'openapi', 'openapi': {'name': 'get_weather', 'description': 'Retrieve weather information for a location', 'spec': {'openapi': '3.1.0', 'info': {'title': 'get weather data', 'description': 'Retrieves current weather data for a location based on wttr.in.', 'version': 'v1.0.0'}, 'servers': [{'url': 'https://wttr.in'}], 'auth': [], 'paths': {'/{location}': {'get': {'description': 'Get weather information for a specific location', 'operationId': 'GetCurrentWeather', 'parameters': [{'name': 'location', 'in': 'path', 'description': 'City or location to retrieve the weather for', 'required': True, 'schema': {'type': 'string'}}, {'name': 'format', 'in': 'query', 'description': 'Always use j1 value for this parameter', 'required': True, 'schema': {'type': 'string', 'default': 'j1'}}], 'responses': {'200': {'description': 'Successful response', 'content': {'text/plain': {'schema': {'type': 'string'}}}}, '404': {'description': 'Location not found'}}, 'deprecated': 

# Create AI Foundry Agent

In [6]:
# Create agent with AI search tool and process assistant run
agent = agents_client.create_agent(
    model=deployment_name,
    name="openapi-agent",
    instructions="You are a helpful assistant",
    tools=openapi.definitions,
    tool_resources = openapi.resources,
)

agent.items

<bound method _MyMutableMapping.items of {'id': 'asst_WeSqfvp4br9fXsRqCVyVW3GX', 'object': 'assistant', 'created_at': 1748700847, 'name': 'openapi-agent', 'description': None, 'model': 'gpt-4o', 'instructions': 'You are a helpful assistant', 'tools': [{'type': 'openapi', 'openapi': {'name': 'get_weather', 'description': 'Retrieve weather information for a location', 'spec': {'openapi': '3.1.0', 'info': {'title': 'get weather data', 'description': 'Retrieves current weather data for a location based on wttr.in.', 'version': 'v1.0.0'}, 'servers': [{'url': 'https://wttr.in'}], 'auth': [], 'paths': {'/{location}': {'get': {'description': 'Get weather information for a specific location', 'operationId': 'GetCurrentWeather', 'parameters': [{'name': 'location', 'in': 'path', 'description': 'City or location to retrieve the weather for', 'required': True, 'schema': {'type': 'string'}}, {'name': 'format', 'in': 'query', 'description': 'Always use j1 value for this parameter', 'required': True, 

# Create the thread and attach a new message to it

In [7]:
# Create a thread
thread = agents_client.threads.create()
print(f"Created thread: {thread}\n")


# Add a user message to the thread
from azure.ai.agents.models import MessageRole

message = agents_client.messages.create(
    thread_id=thread.id, 
    role=MessageRole.USER, 
    content="Quale sarà la temperatura massima a Chiavari il 15 giugno? Se il giorno non è ancora disponibile, indica la temperatura massima del giorno più lontano che sai prevedere.",
)

# let's see the messages associated with the thread
list(agents_client.messages.list(thread_id=thread.id))

Created thread: {'id': 'thread_q2frFLiPvk5Sr4nuXjmsKgU2', 'object': 'thread', 'created_at': 1748700847, 'metadata': {}, 'tool_resources': {}}



[{'id': 'msg_Fc8OG3XYA6rmhwUW2pjoPpI9', 'object': 'thread.message', 'created_at': 1748700847, 'assistant_id': None, 'thread_id': 'thread_q2frFLiPvk5Sr4nuXjmsKgU2', 'run_id': None, 'role': 'user', 'content': [{'type': 'text', 'text': {'value': 'Quale sarà la temperatura massima a Chiavari il 15 giugno? Se il giorno non è ancora disponibile, indica la temperatura massima del giorno più lontano che sai prevedere.', 'annotations': []}}], 'attachments': [], 'metadata': {}}]

# Run the agent synchronously

In [8]:
%%time

# Run the agent
run = agents_client.runs.create_and_process\
    (thread_id=thread.id, agent_id=agent.id)

print(f"Run finished with status: {run.status}.\n\nRun: {run}")

if run.status == "failed":
    # Check if you got "Rate limit is exceeded.", then you want to get more quota
    print(f"Run failed: {run.last_error}")

Run finished with status: RunStatus.COMPLETED.

Run: {'id': 'run_N6fYqpPDw038uFtpVI5O3NTx', 'object': 'thread.run', 'created_at': 1748700849, 'assistant_id': 'asst_WeSqfvp4br9fXsRqCVyVW3GX', 'thread_id': 'thread_q2frFLiPvk5Sr4nuXjmsKgU2', 'status': 'completed', 'started_at': 1748700849, 'expires_at': None, 'cancelled_at': None, 'failed_at': None, 'completed_at': 1748700852, 'required_action': None, 'last_error': None, 'model': 'gpt-4o', 'instructions': 'You are a helpful assistant', 'tools': [{'type': 'openapi', 'openapi': {'name': 'get_weather', 'description': 'Retrieve weather information for a location', 'spec': {'openapi': '3.1.0', 'info': {'title': 'get weather data', 'description': 'Retrieves current weather data for a location based on wttr.in.', 'version': 'v1.0.0'}, 'servers': [{'url': 'https://wttr.in'}], 'auth': [], 'paths': {'/{location}': {'get': {'description': 'Get weather information for a specific location', 'operationId': 'GetCurrentWeather', 'parameters': [{'name': '

# Fetch messages from the thread after the agent run execution

In [9]:
from azure.ai.agents.models import (MessageTextContent, MessageImageFileContent, MessageTextFileCitationAnnotation, \
    MessageTextFilePathAnnotation, MessageTextUrlCitationAnnotation)

image_files = []
annotations = []
citations = []
citation_annotations = []

if run.status == 'completed':
    messages = agents_client.messages.list(thread_id=thread.id)
    messages_list = list(agents_client.messages.list(thread_id=thread.id))  # Convert iterator to a list
    messages_nr = len(messages_list)
    print(f"Here are the {messages_nr} messages:\n")
    
    for i, message in enumerate(reversed(messages_list), 1):
        j = 0
        print(f"\n===== MESSAGE {i} =====")
        for c in message.content:
            j +=1
            if (type(c) is MessageTextContent):
                print(f"\nMessage {i} / CONTENT {j} (MessageTextContent) --> Text: {c.text.value}")
                for a in c.text.annotations:
                    if type(a) is MessageTextFileCitationAnnotation:
                        print(f">>> Citation in MessageTextContent {j} of message {i}: {a}\n")
                        citations.append(a)
                    elif type(a) is MessageTextFilePathAnnotation:
                        print(f">>> Annotation in MessageTextContent {j} of message {i}: {a}\n")
                        annotations.append(a)
                    elif type(a) is MessageTextUrlCitationAnnotation:
                        print(f">>> Annotation in MessageTextContent {j} of message {i}: {a}\n")
                        citation_annotations.append(a)
                    else:
                        print(f"Unknown type {type(a)}")
            elif (type(c) is MessageImageFileContent):
                print(f"\nMessage {i} / CONTENT {j} (MessageImageFileContent) --> image_file id: {c.image_file.file_id}")
                image_files.append(c.image_file.file_id)
            else:
                print(f"Unknown type {type(a)}")

else:
    print(f"Sorry, I can't proceed because the run status is {run.status}")

Here are the 2 messages:


===== MESSAGE 1 =====

Message 1 / CONTENT 1 (MessageTextContent) --> Text: Quale sarà la temperatura massima a Chiavari il 15 giugno? Se il giorno non è ancora disponibile, indica la temperatura massima del giorno più lontano che sai prevedere.

===== MESSAGE 2 =====

Message 2 / CONTENT 1 (MessageTextContent) --> Text: Al momento, non sono in grado di prevedere il tempo per il 15 giugno. Tuttavia, la previsione più distante disponibile è per il 2 giugno 2025. La temperatura massima prevista a Chiavari per quel giorno è di **23°C**.


In [10]:
# Get the last message from the sender
last_msg = list(agents_client.messages.list(thread_id=thread.id))[0]

if last_msg:
    print(last_msg.content[0].text.value)

Al momento, non sono in grado di prevedere il tempo per il 15 giugno. Tuttavia, la previsione più distante disponibile è per il 2 giugno 2025. La temperatura massima prevista a Chiavari per quel giorno è di **23°C**.


# Print citation_annotations from the messages

In [11]:
print (f"Nr. of citation_annotations: {len(annotations)}\n")

i=0
for ca in citation_annotations:
    i += 1
    print(f"{i} - citation_annotations {ca.text}: {ca.url_citation} (from {ca.start_index} to {ca.end_index})")

Nr. of citation_annotations: 0



# Print annotations from the messages

In [12]:
print (f"Nr. of file path annotations: {len(annotations)}\n")

i=0
for a in annotations:
    i += 1
    print(f"{i} - File annotation paths: {a.text}")
    file_name = a.text.split('/')[-1]
    file_id = a.file_path.file_id

    #agents_client.files.save(file_id=file_id, file_name=file_name)
    project_client.agents.files.save(file_id=file_id, file_name=file_name)
    print(f"\n>>> file <{file_id}> saved as <{file_name}>")
    
    # project_client.agents.save_file(file_id=file_path_annotation.file_path.file_id, file_name=file_name)
    print(f"File annotation {i} saved as file to: {os.getcwd()}/{file_name}")
    image = mpimg.imread(f"{os.getcwd()}/{file_name}") # read the image
    plt.imshow(image)
    plt.show()

Nr. of file path annotations: 0



# Fetch citations from the messages

In [13]:
print (f"Nr. of file path citations: {len(citations)}\n")

i=0
for a in citations:
    i += 1
    print(f"{i} - citation: {a}")

Nr. of file path citations: 0



# Retrieve and download eventual images

In [14]:
print (f"Nr. of image contents: {len(image_files)}\n")

i=0
# Generate an image file for the bar chart
for image_file in image_files:
    i += 1
    print(f"{i} - Image file id: {image_file}")
    file_name = f"{image_file}.png"
    project_client.agents.files.save(file_id=image_file, file_name=file_name)
    print(f"Image content {i} file to: {os.getcwd()}/{file_name}")
    image = mpimg.imread(f"{os.getcwd()}/{file_name}") # read the image
    plt.imshow(image)
    plt.show()

Nr. of image contents: 0



# Run Steps

In [15]:
run_steps = list(agents_client.run_steps.list(run_id=run.id, thread_id=thread.id))

print(f'Nr of run step(s): {len(run_steps)}\n')
i=0
for rs in run_steps:
    i += 1
    print(f"Run step {i}: {rs}", '\n')

Nr of run step(s): 2

Run step 1: {'id': 'step_B9TgPr8NvXp50zs8JBzUsOM3', 'object': 'thread.run.step', 'created_at': 1748700851, 'run_id': 'run_N6fYqpPDw038uFtpVI5O3NTx', 'assistant_id': 'asst_WeSqfvp4br9fXsRqCVyVW3GX', 'thread_id': 'thread_q2frFLiPvk5Sr4nuXjmsKgU2', 'type': 'message_creation', 'status': 'completed', 'cancelled_at': None, 'completed_at': 1748700852, 'expires_at': None, 'failed_at': None, 'last_error': None, 'step_details': {'type': 'message_creation', 'message_creation': {'message_id': 'msg_e70l1ZE98r1mJr1nonQPUQpv'}}, 'usage': {'prompt_tokens': 9250, 'completion_tokens': 58, 'total_tokens': 9308, 'prompt_token_details': {'cached_tokens': 0}}} 

Run step 2: {'id': 'step_1q45EhiPaP4BwooCBIxF1sjM', 'object': 'thread.run.step', 'created_at': 1748700849, 'run_id': 'run_N6fYqpPDw038uFtpVI5O3NTx', 'assistant_id': 'asst_WeSqfvp4br9fXsRqCVyVW3GX', 'thread_id': 'thread_q2frFLiPvk5Sr4nuXjmsKgU2', 'type': 'tool_calls', 'status': 'completed', 'cancelled_at': None, 'completed_at': 

# Collect all resources for this project

In [16]:
all_agents = list_all_agents(client=agents_client)
print(all_agents["summary"])

all_threads = list_all_threads(client=agents_client)
print(all_threads["summary"])

all_files = list_all_files(client=agents_client)
print(all_files["summary"])

all_runs = list_all_runs(client=agents_client)
print(all_runs["summary"])

# all_runsteps=list_all_runsteps(agents_client)
# print(all_runsteps["summary"])

# all_messages = list_all_messages(agents_client)
# print(all_messages["summary"])

all_vectorstores = list_all_vectorstores(client=agents_client)
print(all_vectorstores["summary"])

3 agents
1 threads
0 files
1 runs in 1 threads
0 vector stores


# Teardown for all resources

In [17]:
# delete all vector stores

i=0
for vector_store in all_vectorstores["content"]:
    i += 1
    agents_client.vector_stores.delete(vector_store_id=vector_store.id)
    print(f"{i} - Vector store <{vector_store.id}> has been deleted")

all_vectorstores = list_all_vectorstores(client=agents_client)

print(f"Vector stores deleted: {i}\n")

Vector stores deleted: 0



In [18]:
# delete all files

i=0
for file in all_files['content']:
    i += 1
    agents_client.files.delete(file_id=file.id)
    print(f"{i} - File <{file.filename}> ({file.id}) has been deleted")

all_files = list_all_files(client=agents_client)

print(f"Files deleted: {i}\n")

Files deleted: 0



In [19]:
# delete all threads

i=0
for thread in all_threads["content"]:
    i += 1
    agents_client.threads.delete(thread_id=thread.id)
    print(f"{i} - Thread <{thread.id}> has been deleted")

all_threads = list_all_threads(client=agents_client)

print(f"Threads deleted: {i}\n")

1 - Thread <thread_q2frFLiPvk5Sr4nuXjmsKgU2> has been deleted
Threads deleted: 1



In [20]:
# delete all agents

i=0
for agent in all_agents["content"]:
    i += 1
    agents_client.delete_agent(agent_id=agent.id)
    print(f"{i} - Agent <{agent.id}> has been deleted")

all_agents = list_all_agents(client=agents_client)

print(f"Agents deleted: {i}\n")

1 - Agent <asst_WeSqfvp4br9fXsRqCVyVW3GX> has been deleted
2 - Agent <asst_JtWlXyDdf23Y6syqX8u8pUNv> has been deleted
3 - Agent <asst_Lnu7ZHz95SQfOe9OHBvEKEeA> has been deleted
Agents deleted: 3



# HIC SUNT LEONES